In [38]:
import polars as pl
import sys, os
from sippy import *
import numpy as np
from sippy import functionset as fset
from sippy import functionsetSIM as fsetSIM
import matplotlib.pyplot as plt
import pandas as pd
import control as cnt
import json
from datetime import datetime, timezone
import plotly.graph_objects as go
from plotly_resampler import FigureResampler, FigureWidgetResampler

In [39]:
data_path = '/home/alqua/data/data_vdfs'

In [40]:
start_filter_date = datetime(2024, 1, 9, 00, 00, 0, tzinfo=timezone.utc)
end_filter_date = datetime(2024, 1, 15, 00, 00, 0, tzinfo=timezone.utc)

In [41]:
p1_power = pl.read_parquet(data_path + '/' +'pump1_power_siso.par')
p3_power = pl.read_parquet(data_path + '/' +'pump3_power_siso.par')
p4_power = pl.read_parquet(data_path + '/' +'pump4_power_siso.par')
outflow_df = pl.read_parquet(data_path + '/' +'outflow_miso.par')
pressure_df = pl.read_parquet(data_path + '/' +'pressure_miso.par')


In [42]:
sampling_time = '1s'

p1_power = p1_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p3_power = p3_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

p4_power = p4_power.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )


pressure_df = pressure_df.filter(
                                 (pl.col('time') >= start_filter_date) &
                                   (pl.col('time')<= end_filter_date)
                                 )

In [43]:
sysid_df = outflow_df.join(p1_power, 
                left_on='time', 
                right_on='time').join(p3_power, 
                left_on='time', 
                right_on='time').join(
                p4_power, 
                left_on='time', 
                right_on='time').join(pressure_df, 
                left_on='time', 
                right_on='time').upsample(
                    time_column='time', 
                    every=sampling_time, 
                    maintain_order=True)
                    #).group_by_dynamic(
                    #    'time', 
                    #    every=sampling_time, 
                    #    ).agg(pl.all().mean())

In [44]:
features_cols = {
    'time': 'time_utc',  
    'level': 'h',
    'outflow': 'qout',
    'pump1_speed': 'p1_rpm',
    'pump4_speed': 'p4_rpm',
    'pump3_speed': 'p3_rpm',
    'pump1_power': 'p1_power',
    'pump3_power': 'p3_power',
    'pump4_power': 'p4_power',
    'pressure': 'pressure',
}

sysid_df = (
    sysid_df
    .select(features_cols.keys())
    .rename(features_cols)
    .filter(
        (pl.col("p1_rpm") > 0) |
        (pl.col("p3_rpm") > 0) |
        (pl.col("p4_rpm") > 0)
    )
)




# Exclude rows where all p1_rpm, p3_rpm, p4_rpm are greater than zero
sysid_df = sysid_df.with_columns((
    pl.when(pl.col("p1_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p1"), 
    pl.when(pl.col("p3_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p3"),
    pl.when(pl.col("p4_rpm") > 0).then(pl.col("qout")).otherwise(0).alias("qout_p4")
)).filter(~(
    (pl.col("p1_rpm") > 0) &
    (pl.col("p3_rpm") > 0) &
    (pl.col("p4_rpm") > 0)
))

In [45]:
from sippy import functionset as fset

def identify_system(df, u_col, y_col, test_size=0.6, na=1, nb=2, theta=0, dt=None, predict_test =False, nsteps_ahead=1, plot_results=False, save_tf=False):

    selected_data = df.select([u_col, y_col])
    split_point = int(len(selected_data) * test_size)
    train_df = selected_data.head(split_point)
    test_df = selected_data.tail(len(selected_data) - split_point) 
    


    u_train, u_test = train_df[u_col].to_numpy(), test_df[u_col].to_numpy()
    y_train, y_test = train_df[y_col].to_numpy(), test_df[y_col].to_numpy()
    
    na_ords = [na]         
    nb_ords = [[nb]]       
    theta = [[theta]] 

    id_ARX = system_identification(y_train, u_train, 'ARX', stab_cons=True, 
                                ARX_orders=[na_ords, nb_ords, theta], tsample=dt) 
    
    G = id_ARX.G  
    print(f"\nTransfer function from {u_col} to {y_col}:")
    print("==================")
    print(id_ARX.G)
    if save_tf: 
        tf_data = {
                    "u": u_col,
                    "y": y_col,
                    "na": na, 
                    "nb":nb,
                    "num": [round(x, 5) for x in id_ARX.NUMERATOR[0][0]],
                    "den": [round(x, 5) for x in id_ARX.DENOMINATOR[0][0][1:]],
                    "dt": dt
                }
        
        filename = f"tf_{u_col}_to_{y_col}.json"
        result_path = "results/"
        with open(result_path + filename, 'w') as f:
            json.dump(tf_data, f, indent=4)

    if predict_test: 
        t_test = np.arange(0, len(y_test)) * dt
        Yval = fset.validation(id_ARX, u_test, y_test, t_test, k=nsteps_ahead)
        if plot_results: 
            fig = FigureWidgetResampler(go.Figure())
            fig.update_layout(margin=dict(l=10, r=10, t=10, b=10))
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=y_test,
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.add_trace(
                        go.Scattergl(
                            x=t_test,
                            y=Yval.flatten(),
                            name=f'{y_col} (Predicted from {u_col})',  # Shows column names
                            showlegend=True,
                            mode='lines'
                        )
                    )
            fig.update_layout(height=200, template="plotly_dark")
            display(fig)
            
        
        return id_ARX, G, t_test, Yval

    
    else: 
        return id_ARX, G


In [46]:
columns = [
    ("p1_rpm", "qout_p1"),
    ("p1_rpm", "p1_power"),

    ("p3_rpm", "qout_p3"),
    ("p3_rpm", "p3_power"),

    ("p4_rpm", "qout_p4"),
    ("p4_rpm", "p4_power"),
]


for u_col, y_col in columns:
    print(f"Identifying model {u_col}, {y_col}")
    id_ARX, G, t_test, Yval = identify_system(
        df=sysid_df, 
        u_col=u_col,
        y_col=y_col,
        test_size=0.4, 
        na=1,
        nb=1,
        theta=0,
        dt=1,
        predict_test=True, 
        nsteps_ahead=1, 
        plot_results=True, 
        save_tf=True
    )
    # Process/save result as needed

Identifying model p1_rpm, qout_p1



Transfer function from p1_rpm to qout_p1:

 0.00575
----------
z - 0.9885

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '91a765b3-c5d4-4d65-ad3a-6777c0430eda',
              'x': array([     0,    197,    311, ..., 286890, 287336, 287521]),
              'y': array([566., 522., 617., ..., 863., 819., 878.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'e5b33e85-5632-45ae-a097-444e726ce762',
              'x': array([     0,     18,    312, ..., 286891, 287337, 287521]),
              'y': array([  0.        , 516.42626735, 616.60535337, ..., 859.24800902,
                          815.66968635, 878.84461839])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template

Identifying model p1_rpm, p1_power

Transfer function from p1_rpm to p1_power:

 0.00856
----------
z - 0.8163

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'cc55f2bd-1773-4e1d-8175-a87cb9c45c8f',
              'x': array([     0,    242,    398, ..., 286970, 287451, 287521]),
              'y': array([51.29999924, 58.09999847, 51.20000076, ..., 48.79999924, 45.        ,
                          48.90000153])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '2ac182e5-9828-4c8c-8113-c2dfd70fa1b3',
              'x': array([     0,     58,    474, ..., 286971, 287452, 287521]),
              'y': array([ 0.        , 54.8977723 , 51.30617311, ..., 49.07842549, 45.77115983,
                          48.41350275])}],
    'layout': {'height': 

...

FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '76dba878-235e-4402-8aae-67c3d79e4026',
              'x': array([     0,      1,    289, ..., 286802, 287089, 287521]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'aff14139-703c-4361-8c66-b60bd8fea5f4',
              'x': array([     0,      1,    289, ..., 286802, 287089, 287521]),
              'y': array([0., 0., 0., ..., 0., 0., 0.])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template': '...'}
})

Identifying model p4_rpm, qout_p4

Transfer function from p4_rpm to qout_p4:

 0.007041
----------
z - 0.9865

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'eed1d0bb-bab6-46a8-9d16-6aade53c49ce',
              'x': array([     0,      1,    289, ..., 286890, 287336, 287521]),
              'y': array([  0.,   0.,   0., ..., 863., 819., 878.])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'f9f69bde-946c-4fcf-9b85-4764433b1cfb',
              'x': array([     0,      1,    289, ..., 286891, 287337, 287521]),
              'y': array([  0.        ,   0.        ,   0.        , ..., 858.30859088,
                          814.80603379, 877.84053108])}],
    'layout': {'height': 200, 'margin': {'b': 10, 'l': 10, 'r': 10, 't': 10}, 'template

Identifying model p4_rpm, p4_power

Transfer function from p4_rpm to p4_power:

 0.003599
----------
z - 0.9233

dt = 1



FigureWidgetResampler({
    'data': [{'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': 'cc52348e-4687-4aba-a6db-90f91e9e3741',
              'x': array([     0,      1,    289, ..., 287104, 287130, 287521]),
              'y': array([ 0.        ,  0.        ,  0.        , ..., 45.90000153, 41.40000153,
                          41.        ])},
             {'mode': 'lines',
              'name': ('<b style="color:sandybrown">[R' ... 'style="color:#fc9944">~288</i>'),
              'showlegend': True,
              'type': 'scattergl',
              'uid': '1515b1be-ee30-4e0e-882e-5a7ec0ce6c2c',
              'x': array([     0,      1,    289, ..., 287105, 287131, 287521]),
              'y': array([ 0.        ,  0.        ,  0.        , ..., 45.95569825, 41.75384364,
                          42.16512822])}],
    'layout': {'height': 